In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, argparse, math, random
from datetime import datetime

k = 10

ka_test_folder = "../dataset/Test/ka"
kAA_test_folder = "../dataset/Test/kAA"
kha_test_folder = "../dataset/Test/kha"

ka_train_folder = "../dataset/Train/ka"
kAA_train_folder = "../dataset/Train/kAA"
kha_train_folder = "../dataset/Train/kha"

data = [] # training data

In [2]:
def fileHandle(fileName):
    wholeData = []
    file = open(fileName)
    for line in file:
        teLine = line.rstrip('\n ').split(' ')
        nLine = [float(i) for i in teLine]
        nLine = np.array(nLine)
        wholeData.append(nLine)
    file.close()
    return wholeData

# Reading all the example sets in a folder\n",
def readFolder(folder):
    data = []
    count = 0
    for root, dirs, files in os.walk(folder):
        for f in files:
#             print("Reading: ",f)
            path = os.path.relpath(os.path.join(root, f), ".")
    #         target = os.path.relpath(os.path.join(root, os.path.splitext(f)[0]))
            data.append(fileHandle(path))
            count += 1
    print(count," files read in folder ",folder)
    return data

# Reading all the training data
data.append(readFolder(ka_train_folder))
data.append(readFolder(kAA_train_folder))
data.append(readFolder(kha_train_folder))

383  files read in folder  ../dataset/Train/ka
510  files read in folder  ../dataset/Train/kAA
61  files read in folder  ../dataset/Train/kha


In [36]:
# DTW
def dtw(vector1, vector2):
    tm = len(vector1)
    tn = len(vector2)
    
#     matrix = [[0 for x in range(tn)] for y in range(tm)]
    matrix = np.zeros(shape=(tm,tn), dtype=np.float64)
    for i in range(tm):
        matrix[i][0] = float("inf")
    for i in range(tn):
        matrix[0][i] = float("inf")
    
    matrix[0][0] = 0
    
    for i in range(tm):
        for j in range(tn):
            cost = np.linalg.norm(vector1[i] - vector2[j])
            cost *= cost  
            matrix[i][j] = cost + min(matrix[i-1][j-1], min(matrix[i-1][j], matrix[i][j-1]))
#     print("matrix: ",matrix)
    ans = (matrix[tm-1][tn-1])/(tm*tn)
#     print("DTW value: ",ans)
    return ans
    
def knn(test):
    dataSize = len(data[0])+len(data[1])+len(data[2])
#     print("data size: ",dataSize)
    dtwArray = np.zeros(shape=(dataSize,2), dtype=np.float64)
#     print("array: ",dtwArray)
    
    count = 0
    
    for i in range(len(data[0])):
        dtwArray[count,0] = dtw(test,data[0][i])
        dtwArray[count,1] = 0
        count += 1
    for i in range(len(data[1])):
        dtwArray[count,0] = dtw(test,data[1][i])
        dtwArray[count,1] = 1
        count += 1
    for i in range(len(data[2])):
        dtwArray[count,0] = dtw(test,data[2][i])
        dtwArray[count,1] = 2
        count += 1
        
    dtwArray = dtwArray[np.argsort(dtwArray[:, 0])]
#     dtwArray = np.sort(dtwArray,axis=0) #########

    freq = np.zeros(3)
    
    for i in range(k):
#         print("dtwArray[i,1] Value: ",dtwArray[i,1])
        freq[int(dtwArray[i,1])] += 1
#     print("Freq: ", freq)
    ans = np.argmax(freq)
    return 

def classifySet(test):
    decision = np.zeros(3)
    
    count = 0
    for example in test:
        print("Processing test set no: ",count)
        count += 1
        decision[knn(example)] += 1
#         break
    return decision

In [ ]:
confusion = []

print("confusing ka")
confusion.append(classifySet(readFolder(ka_test_folder)))
print("confusing kAA")
confusion.append(classifySet(readFolder(kAA_test_folder)))
print("confusing kha")
confusion.append(classifySet(readFolder(kha_test_folder)))
print("Confusion Matrix")
print(confusion)

confusing ka
96  files read in folder  ../dataset/Test/ka
Processing test set no:  0
Processing test set no:  1
Processing test set no:  2
Processing test set no:  3
Processing test set no:  4
Processing test set no:  5
